In [12]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pymysql
import re
from urllib.parse import urlparse, parse_qs

In [16]:
# mysql db connect
# need to fill below host, user, password
#db = pymysql.connect(host='', user='', password='', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

In [5]:
# ------ 1. 연재 중 웹툰
#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"

# 요일 탭 클릭
def setDay(dayvalue):
    day = browser.find_element_by_xpath("//button[@data-tab='" + str(dayvalue) + "']")
    day.click()

# 제목 추출
def day_webtoon(day, num):
    # 작품 불러오기
    webtoon = browser.find_element_by_xpath("//*[@id=\"scheduled-day-" + str(day) + "\"]/li[" + str(num) + "]/a")
    
    # url
    toon_url = webtoon.get_attribute('href')
    
    # ID
    toon_id = 'lezhin_' + str(toon_url).replace("https://www.lezhin.com/ko/comic/", '')
    
    # 요일 추출
    if day == 1: weekday = 'mon'
    if day == 2: weekday = 'tue'
    if day == 3: weekday = 'wed'
    if day == 4: weekday = 'thu'
    if day == 5: weekday = 'fri'
    if day == 6: weekday = 'sat'
    if day == 0: weekday = 'sun'
    cursor1.execute(weekday_sql, (toon_id, weekday))
    
    # 제목
    toon_name = webtoon.get_attribute('data-ga-event-label')
    # 작가
    writers = webtoon.find_element_by_class_name('lzComic__artist')
    wrt_name = writers.text
    # 장르
    genres = webtoon.find_element_by_class_name('lzComic__genre')
    genre = genres.text
    
    toon_desc = ""
    
    # 썸네일 주소
    webtoon = browser.find_element_by_xpath("//*[@id=\"scheduled-day-" + str(day) + "\"]/li[" + str(num) + "]/a/div[1]/picture/source")
    if webtoon.get_attribute('srcset') != "//cdn.lezhin.com/files/assets/blank.png":
          toon_thumb_url = webtoon.get_attribute('srcset')
    else: toon_thumb_url = webtoon.get_attribute('data-srcset')
    
    #print(toon_name)
    
    # toon_info
    cursor1.execute(sel_sql)
    result = cursor1.fetchall()
    if (toon_id not in result):
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'lezhin', wrt_name, 'X', toon_thumb_url))
        cursor3.execute(genre_sql, (toon_id, genre))
    
    db.commit()
    

for day in range(0, 7):
    
    browser.get('https://www.lezhin.com/ko/scheduled')
    setDay(day)
    browser.implicitly_wait(3)
    
    print('*********************')
    print(day)
    
    for num in range(1, 100):
        try:
            day_webtoon(day, num)
        except:
            break;

*********************
0
*********************
1


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_20_nov' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_20_nov-일상' for key 'PRIMARY'")
  self._do_get_result()


*********************
2


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_baquimentary' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_baquimentary-일상' for key 'PRIMARY'")
  self._do_get_result()


*********************
3
*********************
4


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_friend' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_friend-로맨스' for key 'PRIMARY'")
  self._do_get_result()


*********************
5


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_cat_fist' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_cat_fist-일상' for key 'PRIMARY'")
  self._do_get_result()


*********************
6


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_parkxkim' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'lezhin_parkxkim-BL' for key 'PRIMARY'")
  self._do_get_result()


In [16]:
# ------ 2. 완결 웹툰 
#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"

# 장르 탭 클릭
# genrevalue: 2~11
def setGenre(genrevalue):
    genre = browser.find_element_by_xpath("//*[@id=\"completed-tab-list\"]/button[" + str(genrevalue) + "]")
    genre.click()
    

# 제목 추출
def finished_webtoon(genre, num):
    
    # 작품 불러오기
    webtoon = browser.find_element_by_xpath("//*[@id=\"completed-genre\"]/li[" + str(num) + "]/a")
    browser.execute_script("arguments[0].scrollIntoView(true);", webtoon)
    
    # url
    toon_url = webtoon.get_attribute('href')
    
    # ID
    toon_id = 'lezhin_' + str(toon_url).replace("https://www.lezhin.com/ko/comic/", '')
    
    # 장르 설정
    if genre == 2: toon_genre = '로맨스'
    if genre == 3: toon_genre = 'BL'
    if genre == 4: toon_genre = '드라마'
    if genre == 5: toon_genre = '판타지'
    if genre == 6: toon_genre = '개그'
    if genre == 7: toon_genre = '액션'
    if genre == 8: toon_genre = '학원'
    if genre == 9: toon_genre = '미스터리'
    if genre == 10: toon_genre = '일상'
    if genre == 11: toon_genre = '백합'
    
    # 제목
    toon_name = webtoon.get_attribute('data-ga-event-label')
    
    # 작가
    writers = webtoon.find_element_by_class_name('lzComic__artist')
    wrt_name = writers.text
    
    # 썸네일 주소
    thumb = webtoon.find_element_by_xpath("div[1]/picture/source")
    toon_thumb_url = '//cdn.lezhin.com/files/assets/blank.png'
    while toon_thumb_url == '//cdn.lezhin.com/files/assets/blank.png':
        toon_thumb_url = thumb.get_attribute('srcset')
    
    toon_desc = ""

    # print('---')
    
    # toon_info
    cursor1.execute(sel_sql)
    result = cursor1.fetchall()
    if (toon_id not in result):
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'lezhin', wrt_name, 'O', toon_thumb_url))
        cursor3.execute(genre_sql, (toon_id, toon_genre))
        db.commit()

browser.get('https://www.lezhin.com/ko/completed')
browser.implicitly_wait(3)

# 페이지 읽어가며 작품 읽는 루프
for genre in range(2, 12):
    print('*********************')
    print(genre)
    page = 0
    
    setGenre(genre)
    browser.implicitly_wait(3)
    
    browser.get('https://www.lezhin.com/ko/completed?page=' + str(page))
    
    
    num = 1
    while num < 99999:
        # print(num)
        try:
            finished_webtoon(genre, num)
            num += 1
        except:
            # 다음 페이지가 유효한 지 검사
            browser.get('https://www.lezhin.com/ko/completed?page=' + str(page+1))
            num = 1
            try:
                browser.find_element_by_xpath("//*[@id=\"completed-genre\"]/li[" + str(num) + "]")
                page += 1
                continue
            except NoSuchElementException:
                break


*********************
2
*********************
3
*********************
4
*********************
5
*********************
6
*********************
7
*********************
8
*********************
9
*********************
10
*********************
11


In [19]:
# ------ 3. 회차 정보
# 창 제일 위에 띄워놔야 함 안 그러면 읽는 거 포기하고 일시정지 상태됨 ㅋㅋ

#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "lezhin"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

desc_sql = "update toon_info set toon_desc = %s where toon_id = %s and toon_desc = ''"

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url']
    
    #만약 누락 된 에피소드가 있다면 아래 코드 이용해서 알파벳 순 아이디 분류 후 적용
    '''
    if "lezhin_p" in toon_id:
        pass
    elif "lezhin_q" in toon_id:
        pass
    elif "lezhin_r" in toon_id:
        pass
    elif "lezhin_s" in toon_id:
        pass
    elif "lezhin_t" in toon_id:
        pass
    elif "lezhin_u" in toon_id:
        pass
    elif "lezhin_v" in toon_id:
        pass
    elif "lezhin_w" in toon_id:
        pass
    elif "lezhin_x" in toon_id:
        pass
    elif "lezhin_y" in toon_id:
        pass
    elif "lezhin_z" in toon_id:
        pass
    else:
        continue
    '''
    
    browser.get(toon_url)
    
    #print('*********************************************')
    #print('<' + toon_id + '>')
    
    # 설명
    description = browser.find_element_by_xpath("//*[@id=\"episode-side\"]/section/section[2]/p")
    toon_desc = description.text
    cursor3.execute(desc_sql,(toon_desc, toon_id))
    
    num = 1 # episode 개수 탐색
    while num < 9999:
        try:
            browser.get(toon_url)
            episode = browser.find_element_by_xpath("//*[@id=\"comic-episode-list\"]/li[" + str(num) + "]/button")
            browser.execute_script("arguments[0].scrollIntoView(true);", episode) # Scroll to this episode
            #browser.implicitly_wait(3)
            
            epi_id = episode.get_attribute('data-episode-id')
            epi_type = episode.get_attribute('data-episode-type')
            epi_title = episode.find_element_by_xpath("//*[@id=\"" + epi_type + epi_id + "\"]/div[1]").text
            
            # 에피소드 제목
            epi_name = episode.find_element_by_xpath("//*[@id=\"" + epi_type + epi_id + "\"]/div[2]").text
            
            # 썸네일 링크
            thumb = episode.find_element_by_xpath("//*[@id=\"comic-episode-list\"]/li[" + str(num) + "]/button/div[1]/picture/source")
            epi_thumb_url = '//cdn.lezhin.com/files/assets/blank.png'
            while epi_thumb_url == '//cdn.lezhin.com/files/assets/blank.png':
                epi_thumb_url = thumb.get_attribute('srcset')
            
            # 업데이트 날짜 추출
            date = '20' + episode.find_element_by_xpath("//*[@id=\"" + epi_type + epi_id + "\"]/div[3]").text
            epi_date = date.replace(".", "-")
            
             # 에피소드 링크
            episode.click()
            epi_url = browser.current_url
            
            cursor2.execute(sel_dup_sql, (toon_id, epi_name))
            if cursor2.fetchall():
                pass
            else:
                cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                db.commit()
            
            num += 1
            #print('---')
        except:
            break
#db.close()

In [ ]:
### 무료 웹툰 페이지에 안 긁혀온 작품도 있던데 나중에 시간나면 확인해보삼...